In [1]:
#dependencies

from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import os

import seaborn as sns
%matplotlib inline

# import datetime
# from dateutil.relativedelta import relativedelta
# from datetime import date

#import re
#import nltk
#from nltk.corpus import stopwords


In [2]:
# Read in csv's

temp_us = "US_youtube_trending_data.csv"
#temp_br = "..//practice_sources/BR_youtube_trending_data.csv"
#temp_ca = "..//practice_sources/CA_youtube_trending_data.csv"
#temp_de = "..//practice_sources/DE_youtube_trending_data.csv"
#temp_fr = "..//practice_sources/FR_youtube_trending_data.csv"
#temp_gb = "..//practice_sources/GB_youtube_trending_data.csv"
#temp_in = "..//practice_sources/IN_youtube_trending_data.csv"
#temp_jp = "..//practice_sources/JP_youtube_trending_data.csv"
#temp_kr = "..//practice_sources/KR_youtube_trending_data.csv"
#temp_mx = "..//practice_sources/MX_youtube_trending_data.csv"
#temp_ru = "..//practice_sources/RU_youtube_trending_data.csv"

us_df = pd.read_csv(temp_us)
#br_df = pd.read_csv(temp_br)
#ca_df = pd.read_csv(temp_ca)
#de_df = pd.read_csv(temp_de)
#fr_df = pd.read_csv(temp_fr)
#gb_df = pd.read_csv(temp_gb)
#in_df = pd.read_csv(temp_in)
#jp_df = pd.read_csv(temp_jp)
#kr_df = pd.read_csv(temp_kr)
#mx_df = pd.read_csv(temp_mx)
#ru_df = pd.read_csv(temp_ru)



In [3]:
# Updating categoryId column with corresponding string for all csv's

category_codes = {
    "1":'Film & Animation',
    "2":'Autos & Vehicles',
    "10":'Music',
    "15":'Pets & Animals',
    "17":'Sports',
    "18":'Short Movies',
    "19":'Travel & Events',
    "20":'Gaming',
    "21":'Videoblogging',
    "22":'People & Blogs',
    "23":'Comedy',
    "24":'Entertainment',
    "25":'News & Politics',
    "26":'Howto & Style',
    "27":'Education',
    "28":'Science & Technology',
    "29":'Nonprofits & Activism',
    "30":'Movies',
    "31":'Anime/Animation',
    "32":'Action/Adventure',
    "33":'Classics',
    "34":'Comedy',
    "35":'Documentary',
    "36":'Drama',
    "37":'Family',
    "38":'Foreign',
    "39":'Horror',
    "40":'Sci-Fi/Fantasy',
    "41":'Thriller',
    "42":'Shorts',
    "43":'Shows',
    "44":'Trailers'
}

cat_code = us_df["categoryId"].map(lambda x: category_codes[str(x)])
us_df['cat_codes'] = cat_code

#cat_code = br_df["categoryId"].map(lambda x: category_codes[str(x)])
#br_df['cat_codes'] = cat_code

#cat_code = ca_df["categoryId"].map(lambda x: category_codes[str(x)])
#ca_df['cat_codes'] = cat_code

#cat_code = de_df["categoryId"].map(lambda x: category_codes[str(x)])
#de_df['cat_codes'] = cat_code

#cat_code = fr_df["categoryId"].map(lambda x: category_codes[str(x)])
#fr_df['cat_codes'] = cat_code

#cat_code = gb_df["categoryId"].map(lambda x: category_codes[str(x)])
#gb_df['cat_codes'] = cat_code

#cat_code = in_df["categoryId"].map(lambda x: category_codes[str(x)])
#in_df['cat_codes'] = cat_code

#cat_code = jp_df["categoryId"].map(lambda x: category_codes[str(x)])
#jp_df['cat_codes'] = cat_code

#cat_code = kr_df["categoryId"].map(lambda x: category_codes[str(x)])
#kr_df['cat_codes'] = cat_code

#cat_code = mx_df["categoryId"].map(lambda x: category_codes[str(x)])
#mx_df['cat_codes'] = cat_code

#cat_code = ru_df["categoryId"].map(lambda x: category_codes[str(x)])
#ru_df['cat_codes'] = cat_cod#e

In [4]:
# Add country string column

us_df['country'] = 'United States'
#br_df['country'] = 'Brazil'
#ca_df['country'] = 'Canada'
#de_df['country'] = 'Germany'
#fr_df['country'] = 'France'
#gb_df['country'] = 'United Kingdom'
#in_df['country'] = 'India'
#jp_df['country'] = 'Japan'
#kr_df['country'] = 'South Korea'
#mx_df['country'] = 'Mexico'
#ru_df['country'] = 'Russia'

In [5]:
#concat all 11 csv's
ind_dfs = [us_df]
# ind_dfs = [us_df, br_df, ca_df, de_df, fr_df, gb_df, in_df, jp_df, kr_df, mx_df, ru_df]
main_df = pd.concat(ind_dfs)

In [6]:
# remove unwanted pipes from tags with .split()

def split_tags(title):
    if title == '[none]': return []
    return title.lower().replace('"','').split('|')

main_df['title'] = main_df['title'].apply(split_tags)

In [7]:
# add trending days column

temp_df = main_df.groupby('video_id')['trending_date'].count().reset_index()
temp_df.rename({'trending_date':'trending_days'}, axis = 'columns', inplace = True)
main_df = main_df.merge(temp_df, on = 'video_id', how = 'inner').sort_values('trending_date')

In [8]:
main_df = main_df.drop_duplicates(subset = 'video_id', keep='last')

In [23]:
main_df['tags'].value_counts()

[None]                                                                                                                                                                                                                                                                                                                                                                                                                                 3547
Amber|amber vtuber|genshi|genshi game|genshi impact|genshi video|genshin|genshin game|genshin impact|genshin impact 2020|genshin impact game|genshin impact good|genshin impact graphics|genshin impact introduction|genshin impact manga|genshin impact wiki|geshin|geshin game|Teyvat|yuanshen game|miHoYo China|miHoYo Japan|adventure story|open world game|anime style|MMORPG|anime games|mobile game|MMO PlayStation|yt:cc=on      40
life hacks|crafts|slime|do it yourself|diy|lifehacks|dily projects|useful things|how to|experiment|experiments|diy activities|handcraft|prank|fu

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
main_df['category_e'] = (encoder.fit_transform(main_df.cat_codes.values)).astype(float)

In [10]:
main_df['target'] = (main_df['trending_days'] > 4).astype(int)

In [11]:
main_df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,cat_codes,country,trending_days,category_e,target
2098,3-144RjSRWI,"[pokã©mon: twilight wings , episode 7 , sky]",2020-08-06T13:00:20Z,UCFctpiB_Hnlk3ejWfHqSm6Q,The Official PokÃ©mon YouTube channel,20,2020-08-12T00:00:00Z,PokÃ©mon: Twilight Wings|Pokemon|Twilight Wings,1099782,64975,...,9386,https://i.ytimg.com/vi/3-144RjSRWI/default.jpg,False,False,Itâ€™s all come down to this. Check out the fi...,Gaming,United States,1,5.0,0
2099,eQhWGnKw5Hc,[boosie badazz & mo3 - mop wit it (official vi...,2020-08-06T01:22:42Z,UC_PQ7gGEKIwJv0YsnxH3dUw,HSM,10,2020-08-12T00:00:00Z,Mo3|Dallas|Texas|Rap Music|Badazz,451058,16636,...,651,https://i.ytimg.com/vi/eQhWGnKw5Hc/default.jpg,False,False,From the album Badazz MO3. Out now!Stream: htt...,Music,United States,1,7.0,0
2100,7QJPCIBgV5g,[cinemassacre channel update 2020],2020-08-07T16:00:04Z,UC0M0rxSz3IF0CsSour1iWmw,Cinemassacre,22,2020-08-12T00:00:00Z,Cinemassacre Channel Update 2020|Cinemassacre|...,403592,39766,...,3608,https://i.ytimg.com/vi/7QJPCIBgV5g/default.jpg,False,False,Cinemassacre Channel Update 2020 Recap: I hope...,People & Blogs,United States,1,10.0,0
2101,Ea04DriyYb4,[video shows beirut blast as bride poses on he...,2020-08-06T00:00:01Z,UCYUbNjkuE4lsr2v1Id2O1oA,ABC7 News Bay Area,25,2020-08-12T00:00:00Z,[None],1793900,14805,...,4037,https://i.ytimg.com/vi/Ea04DriyYb4/default.jpg,False,False,Video captured the terrifying moment when a br...,News & Politics,United States,1,8.0,0
2102,Qn8dMQJlDZE,[if 2020 gave us zombies instead of a pandemic],2020-08-06T14:42:50Z,UCh9IfI45mmk59eDvSWtuuhQ,Ryan George,23,2020-08-12T00:00:00Z,ryan george|comedy|funny|sketch|pandemic|zombi...,531928,69217,...,6823,https://i.ytimg.com/vi/Qn8dMQJlDZE/default.jpg,False,False,Hi there hello please subscribe so I can feed ...,Comedy,United States,1,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118955,Lz5_WqwUAg0,[pat mcafee reacts to the colts trading for ma...,2022-03-21T18:10:56Z,UCxcTeAKWJca6XyJ37_ZoKIQ,The Pat McAfee Show,17,2022-03-26T00:00:00Z,pat mcafee|podcast|sports|nfl|ncaa|football|fo...,714487,12280,...,1926,https://i.ytimg.com/vi/Lz5_WqwUAg0/default.jpg,False,False,Colts just won the Super Bowl.This is a conver...,Sports,United States,4,13.0,0
118959,0DSO5Khbi1s,[confidence is a cult],2022-03-21T17:00:19Z,UCoOss5XiPpnLHGmLrBvNkJg,Mina Le,26,2022-03-26T00:00:00Z,confidence|culure|video essay|analysis|mina le...,584280,48253,...,2374,https://i.ytimg.com/vi/0DSO5Khbi1s/default.jpg,False,False,Use code MINA16 for up to 16 FREE MEALS + 3 Su...,Howto & Style,United States,4,6.0,0
118965,1lIi1txDVzo,"[$1 vs $29,000 house!]",2022-03-23T18:00:00Z,UCIPPMRA040LQr5QPyJEbmXA,MrBeast Gaming,20,2022-03-26T00:00:00Z,[None],10574703,537124,...,22158,https://i.ytimg.com/vi/1lIi1txDVzo/default.jpg,False,False,Get my Shrek x MrBeast merch before itâ€™s go...,Gaming,United States,3,5.0,0
118915,E_XbCf9W2Pk,[trying $1 cheapest nyc food!ðÿ¤‘ #shorts #nyc],2022-03-22T02:19:36Z,UCDy3PfaFLvnXfN-NTYKk9gQ,Gianna Christine,26,2022-03-26T00:00:00Z,[None],364678,27436,...,310,https://i.ytimg.com/vi/E_XbCf9W2Pk/default.jpg,False,False,NaN,Howto & Style,United States,4,6.0,0


In [12]:
############ dataframe for tags nlp model #######################
# add country when needed
tags_df = main_df[['target', 'cat_codes', 'trending_days', 'tags','category_e']].reset_index()
tags_df

,index,target,cat_codes,trending_days,tags,category_e
0,2098,0,Gaming,1,PokÃ©mon: Twilight Wings|Pokemon|Twilight Wings,5.0
1,2099,0,Music,1,Mo3|Dallas|Texas|Rap Music|Badazz,7.0
2,2100,0,People & Blogs,1,Cinemassacre Channel Update 2020|Cinemassacre|...,10.0
3,2101,0,News & Politics,1,[None],8.0
4,2102,0,Comedy,1,ryan george|comedy|funny|sketch|pandemic|zombi...,1.0
...,...,...,...,...,...,...
21345,118955,0,Sports,4,pat mcafee|podcast|sports|nfl|ncaa|football|fo...,13.0
21346,118959,0,Howto & Style,4,confidence|culure|video essay|analysis|mina le...,6.0
21347,118965,0,Gaming,3,[None],5.0
21348,118915,0,Howto & Style,4,[None],6.0


In [13]:
tags_df.to_csv('tags_df5.csv')

In [14]:
tags_df.count()

index            21350
target           21350
cat_codes        21350
trending_days    21350
tags             21350
category_e       21350
dtype: int64